In [1]:
from __future__ import division

import pandas as pd
import numpy as np
import math
from datetime import datetime,timedelta

%matplotlib inline

In [7]:
training_path = 'dataSets/training/'

In [9]:
training_traj = pd.read_csv(training_path+'trajectories(table 5)_training.csv')

In [16]:
def time_to_window(x):
    dt = datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    dtmin = int(dt.minute / 20) * 20
    dtwindow = datetime(dt.year, dt.month, dt.day, dt.hour, dtmin, 0)
    return dtwindow

In [20]:
#training_traj['starting_datetime'] = pd.to_datetime(training_path['starting_time'])
training_traj['window'] = training_traj['starting_time'].apply(time_to_window)

In [68]:
training_traj_group_mean = \
training_traj.groupby(['intersection_id','tollgate_id','window']).mean()['travel_time']

In [185]:
training_traj_group_count = \
training_traj.groupby(['intersection_id','tollgate_id','window']).count()['travel_time']

In [28]:
ts = training_traj['travel_seq'][0]

In [29]:
ts

'105#2016-07-19 00:14:24#9.56;100#2016-07-19 00:14:34#6.75;111#2016-07-19 00:14:41#13.00;103#2016-07-19 00:14:54#7.47;122#2016-07-19 00:15:02#32.85'

In [50]:
from datetime import date, time

In [47]:
date_list = []

d1 = date(2016, 7, 19)  # start date
d2 = date(2016, 10, 17)  # end date

delta = d2 - d1         # timedelta

for i in range(delta.days + 1):
    date_list.append(d1 + timedelta(days=i))

In [52]:
give_time_list1 = [time(6,0), time(6,20), time(6,40), 
                   time(7,0), time(7,20), time(7,40)]
pred_time_list1 = [time(8,0), time(8,20), time(8,40), 
                   time(9,0), time(9,20), time(9,40)]

In [205]:
1/2.71828

0.36787968862663156

In [203]:
A2m.index[300].weekday()

5

In [204]:
training_traj_group_count.loc['A'].loc[2][260:]

window
2016-07-22 18:40:00    11
2016-07-22 19:00:00    16
2016-07-22 19:20:00    12
2016-07-22 19:40:00     9
2016-07-22 20:00:00     6
2016-07-22 20:20:00    10
2016-07-22 20:40:00     8
2016-07-22 21:00:00     6
2016-07-22 21:20:00    11
2016-07-22 21:40:00     1
2016-07-22 22:00:00     2
2016-07-22 22:20:00     3
2016-07-22 22:40:00     4
2016-07-22 23:00:00     9
2016-07-22 23:20:00     1
2016-07-23 06:00:00    12
2016-07-23 06:20:00    12
2016-07-23 06:40:00    15
2016-07-23 07:00:00    12
2016-07-23 07:20:00     9
2016-07-23 07:40:00    11
2016-07-23 08:00:00     8
2016-07-23 08:20:00    12
2016-07-23 08:40:00    11
2016-07-23 09:00:00    17
2016-07-23 09:20:00    18
2016-07-23 09:40:00    26
2016-07-23 10:00:00    17
2016-07-23 10:20:00     9
2016-07-23 11:00:00     1
                       ..
2016-10-17 14:00:00    20
2016-10-17 14:20:00     6
2016-10-17 14:40:00    11
2016-10-17 15:00:00    12
2016-10-17 15:20:00    15
2016-10-17 15:40:00    12
2016-10-17 16:00:00    11
2016-

In [139]:
A2m = training_traj_group_mean.loc['A'].loc[2]

In [140]:
def interpolate_missing_window(x):
    ss = [(i-datetime(1970,1,1)).total_seconds() for i in x.index]
    tt = x.values
    ssall = np.linspace(ss[0], ss[-1], (ss[-1]-ss[0])/1200+1)
    print(len(ss), len(tt), len(ssall))
    ttall = np.interp(ssall, ss, tt)
    ssall = [int(i) for i in ssall]
    ssall = [datetime(1970,1,1)+timedelta(seconds=i) for i in ssall]
    return pd.Series(index=ssall, data=ttall)

In [141]:
A2m = interpolate_missing_window(A2m)

(5965, 5965, 6551)


In [145]:
morning_train66 = []
for i in date_list:
    tmp = []
    for j in give_time_list1:
        tmp.append(A2m[datetime.combine(i, j)])
    for j in pred_time_list1:
        tmp.append(A2m[datetime.combine(i, j)]) 
    morning_train66.append(tmp)

In [149]:
morning_train66[0]

[46.356000000000009,
 48.588000000000001,
 66.642499999999998,
 64.680999999999997,
 85.676000000000016,
 58.968888888888877,
 81.602857142857147,
 80.207857142857137,
 63.448461538461544,
 78.05117647058826,
 69.038333333333341,
 69.657142857142858]

In [163]:
def pred_loss(x_pred, x_true):
    x_pred = np.array(x_pred)
    x_true = np.array(x_true)
    return (abs(x_pred-x_true)/x_true).mean()

In [209]:
def naive_pred_loss(x):
    first6 = x[:6]
    pred = [np.mean(first6)]*6
    return pred_loss(pred, x[6:])

In [210]:
naive_loss_list = [naive_pred_loss(i) for i in morning_train66]

In [211]:
np.mean(naive_loss_list)

0.24926126851386465

In [18]:
# -*- coding: utf-8 -*-
#!/usr/bin/env python

"""
Objective:
Calculate the average travel time for each 20-minute time window.

"""

# import necessary modules
import math
from datetime import datetime,timedelta

file_suffix = '.csv'
path = 'dataSets/training/'  # set the data directory

def avgTravelTime(in_file):

    out_suffix = '_20min_avg_travel_time'
    in_file_name = in_file + file_suffix
    out_file_name = in_file.split('_')[1] + out_suffix + file_suffix

    # Step 1: Load trajectories
    fr = open(path + in_file_name, 'r')
    fr.readline()  # skip the header
    traj_data = fr.readlines()
    fr.close()
    print(traj_data[0])

    # Step 2: Create a dictionary to store travel time for each route per time window
    travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
    for i in range(len(traj_data)):
        each_traj = traj_data[i].replace('"', '').split(',')
        intersection_id = each_traj[0]
        tollgate_id = each_traj[1]

        route_id = intersection_id + '-' + tollgate_id
        if route_id not in travel_times.keys():
            travel_times[route_id] = {}

        trace_start_time = each_traj[3]
        trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
        time_window_minute = int(trace_start_time.minute / 20) * 20
        start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
        tt = float(each_traj[-1]) # travel time

        if start_time_window not in travel_times[route_id].keys():
            travel_times[route_id][start_time_window] = [tt]
        else:
            travel_times[route_id][start_time_window].append(tt)

    # Step 3: Calculate average travel time for each route per time window
    fw = open(out_file_name, 'w')
    fw.writelines(','.join(['"intersection_id"', '"tollgate_id"', '"time_window"', '"avg_travel_time"']) + '\n')
    for route in travel_times.keys():
        route_time_windows = list(travel_times[route].keys())
        route_time_windows.sort()
        for time_window_start in route_time_windows:
            time_window_end = time_window_start + timedelta(minutes=20)
            tt_set = travel_times[route][time_window_start]
            avg_tt = round(sum(tt_set) / float(len(tt_set)), 2)
            out_line = ','.join(['"' + route.split('-')[0] + '"', '"' + route.split('-')[1] + '"',
                                 '"[' + str(time_window_start) + ',' + str(time_window_end) + ')"',
                                 '"' + str(avg_tt) + '"']) + '\n'
            fw.writelines(out_line)
    fw.close()

def main():

    in_file = 'trajectories(table 5)_training'
    avgTravelTime(in_file)

if __name__ == '__main__':
    main()





"B","3","1065642","2016-07-19 00:14:24","105#2016-07-19 00:14:24#9.56;100#2016-07-19 00:14:34#6.75;111#2016-07-19 00:14:41#13.00;103#2016-07-19 00:14:54#7.47;122#2016-07-19 00:15:02#32.85","70.85"



In [221]:
import httplib, urllib

In [228]:
conn = httplib.HTTPConnection("52.206.6.10",8083)
conn.request("POST", "", 'cpu_load_short,host=server02 value=0.67')
response = conn.getresponse()
response


<httplib.HTTPResponse instance at 0x10997ae18>

In [230]:
response.status

200